# setup & load data

In [1]:
import pandas as pd
import plotly.express as px
from datetime import datetime

In [ ]:
sale_df = pd.read_csv(r"sales_clean.csv")
rent_df = pd.read_csv(r"rent_clean.csv")

In [3]:
sale_df["transaction_date"]= pd.to_datetime(sale_df["sale_date"])
sale_df["commission_payment_date"]= pd.to_datetime(sale_df["commission_payment_date"])
sale_df.rename(columns={"sale_amount":"transaction_amount"}, inplace=True)

In [4]:
rent_df["transaction_date"]= pd.to_datetime(rent_df["agreement_date"])
rent_df["commission_payment_date"]= pd.to_datetime(rent_df["commission_payment_date"])
rent_df["transaction_amount"] = (rent_df["rent_amount"]*rent_df["rent_months"])

In [5]:
rent_df['transaction_type'] = 'rent'
sale_df['transaction_type'] = 'sale'
common_cols = ['property_id','agent_id','owner_id','client_id','transaction_date',"transaction_amount",'commission_payment_date','commission_amount','transaction_type']
transactions_df = pd.concat([rent_df[common_cols], sale_df[common_cols]])

In [6]:
transactions_df.sample(5)

,property_id,agent_id,owner_id,client_id,transaction_date,transaction_amount,commission_payment_date,commission_amount,transaction_type
245,1323,111,411,2000,2024-08-14,366375.87,2024-11-18,10991.28,sale
1428,1721,49,1450,895,2024-05-17,628639.04,2024-12-18,31431.95,sale
93,1223,138,1631,1920,2024-08-18,185545.83,2024-10-12,3710.92,sale
64,1713,233,1275,1138,2024-12-12,9810.00,2024-12-25,981.00,rent
381,785,73,383,1993,2024-08-27,113400.00,2024-09-20,5670.00,rent


In [7]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000 entries, 0 to 1644
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   property_id              3000 non-null   int64         
 1   agent_id                 3000 non-null   int64         
 2   owner_id                 3000 non-null   int64         
 3   client_id                3000 non-null   int64         
 4   transaction_date         3000 non-null   datetime64[ns]
 5   transaction_amount       3000 non-null   float64       
 6   commission_payment_date  3000 non-null   datetime64[ns]
 7   commission_amount        3000 non-null   float64       
 8   transaction_type         3000 non-null   object        
dtypes: datetime64[ns](2), float64(2), int64(4), object(1)
memory usage: 234.4+ KB


In [8]:
today_date = (transactions_df['commission_payment_date'].max()) + pd.Timedelta(days=1)  # تاريخ اليوم الافتراضي

# Owners Segmentation Using RFM Analysis

RFM score for each owner based on their historical transaction data
to measure value and engagement of owners


## Feature engineering: R,F,M

1. Recency --> today - nearest date (Sales + Rent)
2. Frequenc -->  count of transaction (Sales + Rent)
3. Monetary  --> total commission (Sales + Rent)

In [9]:
owner_agg_df = transactions_df.groupby("owner_id").agg(
                                                        last_date =("transaction_date","max"),
                                                        Frequency =("owner_id","count"),
                                                        Monetary =("commission_amount","sum")
).reset_index()

In [10]:
owner_agg_df['Recency'] = (today_date - owner_agg_df['last_date']).dt.days

In [11]:
owner_agg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1447 entries, 0 to 1446
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   owner_id   1447 non-null   int64         
 1   last_date  1447 non-null   datetime64[ns]
 2   Frequency  1447 non-null   int64         
 3   Monetary   1447 non-null   float64       
 4   Recency    1447 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 56.7 KB


In [12]:
owner_agg_df.head()

,owner_id,last_date,Frequency,Monetary,Recency
0,1,2024-01-28,4,9893.32,338
1,2,2024-12-02,5,45668.26,29
2,3,2024-09-17,2,31257.00,105
3,4,2024-09-23,3,28490.12,99
4,5,2024-02-19,1,14119.14,316


In [13]:
owner_agg_df.describe()

,owner_id,last_date,Frequency,Monetary,Recency
count,1447.000000,1447,1447.000000,1447.000000,1447.000000
mean,877.626123,2024-03-12 07:14:53.158258688,2.073255,24645.467386,293.697996
min,1.000000,2020-06-07 00:00:00,1.000000,438.000000,1.000000
25%,438.500000,2023-10-28 00:00:00,1.000000,8146.625000,62.000000
50%,885.000000,2024-07-05 00:00:00,2.000000,16890.370000,179.000000
75%,1313.500000,2024-10-30 00:00:00,3.000000,33296.800000,430.000000
max,1755.000000,2024-12-30 00:00:00,7.000000,163553.380000,1668.000000
std,507.570847,NaN,1.118261,23211.663082,315.469521


In [14]:
# Recency: most owners are very recent; fewer bins prevent clustering in a single bin -->  3 bins
# Frequency: 4 bins to provide a balanced distribution of owners based on transaction count, separating low and high activity
# Monetary: 4 bins to clearly classify profitability, covering a wide range of low and high values


## RFM Scoring 

Discretization (numerical -->categories"range")

In [15]:
# Recency score (high=old = 1 , low=new = 3)
bins_Recency   = [0, 30, 112, owner_agg_df['Recency'].max()]
labels_Recency = [3, 2, 1]

owner_agg_df['R_Score'] = pd.cut(
    owner_agg_df['Recency'],
    bins=bins_Recency,
    labels=labels_Recency,
    include_lowest=True
)

# Frequency score (low = 1, high= 4)
owner_agg_df['F_Score'] = pd.qcut(owner_agg_df['Frequency'].rank(method='first'), 4, labels=range(1,5))

# Monetary score (low = 1, high= 4)
owner_agg_df['M_Score'] = pd.qcut(owner_agg_df['Monetary'], 4, labels=range(1,5))

Combine R, F, M scores to form a single RFM score "string"

In [16]:
owner_agg_df['RFM_Score'] =( owner_agg_df['R_Score'].astype(str) + 
                             owner_agg_df['F_Score'].astype(str) + 
                             owner_agg_df['M_Score'].astype(str)
)

In [17]:
owner_agg_df.head()

,owner_id,last_date,Frequency,Monetary,Recency,R_Score,F_Score,M_Score,RFM_Score
0,1,2024-01-28,4,9893.32,338,1,4,2,142
1,2,2024-12-02,5,45668.26,29,3,4,4,344
2,3,2024-09-17,2,31257.00,105,2,2,3,223
3,4,2024-09-23,3,28490.12,99,2,3,3,233
4,5,2024-02-19,1,14119.14,316,1,1,2,112


RFM segmentaion:
- Champions: high in all three or at least very recent + (freq or monetary high) (RFM)
- Loyal Owners: frequent and high monetary but not the most recent (F&M) (R1/2)
- High Value – Low Activity: high monetary only (M) (F1/2,R3-1)
- Emerging Owne: recent and good spend but medium frequency (R) (F1-4, M2-4)
- At Risk: very old regardless of frequency or monetary (FM) (R=1)
- Dormant Owner: everyone else (mid-range)

In [18]:
def rfm_owner_segment(row):
    R, F, M = row['R_Score'], row['F_Score'], row['M_Score']

    if R == 3 and F >= 3 and M >= 3:
        return 'Champion'

    elif R >= 2 and F >= 3 and M >= 3:
        return 'Loyal Owner'

    elif R >= 2 and ((F >= 2 and 2 <= M <= 3) or (M >= 2 and 2 <= F <= 3)):
        return 'Emerging Owne'
    
    elif F < 3 and M >= 3:
        return 'High Value-Low Activity'
    
    elif R == 1 and F >= 2 and M >= 2:
        return 'At Risk'

    else:
        return 'Dormant Owner'

owner_agg_df['Segment'] = owner_agg_df.apply(rfm_owner_segment, axis=1)

In [19]:
owner_agg_df.head()

,owner_id,last_date,Frequency,Monetary,Recency,R_Score,F_Score,M_Score,RFM_Score,Segment
0,1,2024-01-28,4,9893.32,338,1,4,2,142,At Risk
1,2,2024-12-02,5,45668.26,29,3,4,4,344,Champion
2,3,2024-09-17,2,31257.00,105,2,2,3,223,Emerging Owne
3,4,2024-09-23,3,28490.12,99,2,3,3,233,Loyal Owner
4,5,2024-02-19,1,14119.14,316,1,1,2,112,Dormant Owner


## visualization

In [20]:
fig_bubble = px.scatter(owner_agg_df,
                        x='Recency', 
                        y='Monetary', 
                        size='Frequency',  
                        color='Segment', 
                        hover_data=['owner_id'], 
                        title='RFM Segmentation Scatter Plot')
fig_bubble.update_layout(xaxis_type="log")
fig_bubble.update_layout(yaxis_type="log")
fig_bubble.show()

In [21]:
"""

Larger dots represent owners with higher transaction frequency.

Champions show high monetary value, frequent transactions, and recent activity, clustering in the top-right of the plot.

Loyal Owners have high frequency and spending but are less recent, clustering in the top-middle.

At Risk owners display high monetary value and frequency but long inactivity, clustering in the top-left.

High Value-Low Activity owners indicate high spending with few transactions.

Dormant Owners show long inactivity or low spending, clustering in the lower area of the plot.

Emerging Owners demonstrate recent activity with moderate to high frequency or spending.

"""

'\n\nLarger dots represent owners with higher transaction frequency.\n\nChampions show high monetary value, frequent transactions, and recent activity, clustering in the top-right of the plot.\n\nLoyal Owners have high frequency and spending but are less recent, clustering in the top-middle.\n\nAt Risk owners display high monetary value and frequency but long inactivity, clustering in the top-left.\n\nHigh Value-Low Activity owners indicate high spending with few transactions.\n\nDormant Owners show long inactivity or low spending, clustering in the lower area of the plot.\n\nEmerging Owners demonstrate recent activity with moderate to high frequency or spending.\n\n'

In [22]:
fig_counts = px.histogram(owner_agg_df, 
                          x='Segment', 
                          title='Number of Owners per Segment', 
                          color='Segment', 
                          text_auto=True)
fig_counts.show()

In [ ]:
"""
Most owners are Dormant or At Risk, with fewer Champions
--> This indicates declining engagement and the need for reactivation strategies to boost loyalty and performance
"""

In [24]:
owner_agg_df[['owner_id', 'R_Score', 'F_Score', 'M_Score', 'Segment']].to_csv(
    "owner_segments.csv", index=False)

# Clients Segmentation Using RFM Analysis

RFM score for each client based on their historical transaction data.



## Feature engineering: R,F,M

1. Recency: How recently a client made a purchase / rent
2. Frequency: How often they made purchases / rent
3. Monetary: How much money they spent

In [25]:
client_agg_df = transactions_df.groupby("client_id").agg(
                                                        last_date =("transaction_date","max"),
                                                        Frequency =("client_id","count"),
                                                        Monetary =("transaction_amount","sum")
).reset_index()

In [26]:
client_agg_df['Recency'] = (today_date - client_agg_df['last_date']).dt.days

In [27]:
client_agg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1598 entries, 0 to 1597
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   client_id  1598 non-null   int64         
 1   last_date  1598 non-null   datetime64[ns]
 2   Frequency  1598 non-null   int64         
 3   Monetary   1598 non-null   float64       
 4   Recency    1598 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 62.6 KB


In [28]:
client_agg_df.head()

,client_id,last_date,Frequency,Monetary,Recency
0,2,2024-05-23,1,256291.77,222
1,4,2024-02-02,1,458279.66,333
2,5,2024-05-09,1,147000.00,236
3,6,2024-06-14,2,168000.00,200
4,7,2024-11-28,3,1043203.93,33


In [29]:
client_agg_df.describe()

,client_id,last_date,Frequency,Monetary,Recency
count,1598.000000,1598,1598.000000,1.598000e+03,1598.000000
mean,1035.235920,2024-02-24 05:53:14.493116672,1.877347,5.643532e+05,310.754693
min,2.000000,2020-05-01 00:00:00,1.000000,8.760000e+03,1.000000
25%,526.250000,2023-10-14 00:00:00,1.000000,1.612962e+05,67.250000
50%,1036.500000,2024-06-25 00:00:00,2.000000,3.537180e+05,189.000000
75%,1541.750000,2024-10-24 18:00:00,2.000000,7.870670e+05,444.000000
max,2060.000000,2024-12-30 00:00:00,8.000000,4.814742e+06,1705.000000
std,590.120765,NaN,1.023200,5.712563e+05,328.276759


## RFM Scoring 

Discretization (numerical -->categories"range")

In [30]:
# Recency score (high=old = 1 , low=new = 3)
bins_Recency   = [0, 30, 224, client_agg_df['Recency'].max()]
labels_Recency = [3, 2, 1]

client_agg_df['R_Score'] = pd.cut(
    client_agg_df['Recency'],
    bins=bins_Recency,
    labels=labels_Recency,
    include_lowest=True
)

# Frequency score (low = 1, high= 4)
client_agg_df['F_Score'] = pd.qcut(client_agg_df['Frequency'].rank(method='first'), 4, labels=range(1,5))

# Monetary score (low = 1, high= 4)
client_agg_df['M_Score'] = pd.qcut(client_agg_df['Monetary'], 4, labels=range(1,5))

Combine R, F, M scores to form a single RFM score "string"

In [31]:
client_agg_df['RFM_Score'] =( client_agg_df['R_Score'].astype(str) + 
                             client_agg_df['F_Score'].astype(str) + 
                             client_agg_df['M_Score'].astype(str)
)

In [32]:
client_agg_df.head()

,client_id,last_date,Frequency,Monetary,Recency,R_Score,F_Score,M_Score,RFM_Score
0,2,2024-05-23,1,256291.77,222,2,1,2,212
1,4,2024-02-02,1,458279.66,333,1,1,3,113
2,5,2024-05-09,1,147000.00,236,1,1,1,111
3,6,2024-06-14,2,168000.00,200,2,2,2,222
4,7,2024-11-28,3,1043203.93,33,2,4,4,244


RFM segmentaion:
- VIP Clients: high in all three or at least very recent + (freq or monetary high) (RFM)
- Regular High-Value: frequent and high monetary but not the most recent (F&M) (R1/2)
- Big Spenders: high monetary only (M) (F1/2,R3-1)
- Potential Loyalists: recent and good spend but medium frequency (R) (F1-4, M2-4)
- At Risk: very old regardless of frequency or monetary (FM) (R=1)
- Mid-Tier: everyone else (mid-range)

In [33]:
def rfm_client_segment(row):
    R, F, M = row['R_Score'], row['F_Score'], row['M_Score']

    if R == 3 and F >= 3 and M >= 3:
        return 'VIP Clients'

    elif R >= 2 and F >= 3 and M >= 3:
        return 'Regular High-Value'

    elif R >= 2 and ((F >= 2 and 2 <= M <= 3) or (M >= 2 and 2 <= F <= 3)):
        return 'Potential Loyalist'
    
    elif F < 3 and M >= 3:
        return 'Big Spender'

    elif R == 1 and F >= 2 and M >= 2:
        return 'At Risk'

    else:
        return 'Mid-Tier'

client_agg_df['Segment'] = client_agg_df.apply(rfm_client_segment, axis=1)

In [34]:
client_agg_df.head()

,client_id,last_date,Frequency,Monetary,Recency,R_Score,F_Score,M_Score,RFM_Score,Segment
0,2,2024-05-23,1,256291.77,222,2,1,2,212,Mid-Tier
1,4,2024-02-02,1,458279.66,333,1,1,3,113,Big Spender
2,5,2024-05-09,1,147000.00,236,1,1,1,111,Mid-Tier
3,6,2024-06-14,2,168000.00,200,2,2,2,222,Potential Loyalist
4,7,2024-11-28,3,1043203.93,33,2,4,4,244,Regular High-Value


## visualization

In [35]:
fig_bubble = px.scatter(client_agg_df,
                        x='Recency', 
                        y='Monetary', 
                        size='Frequency',  
                        color='Segment', 
                        hover_data=['client_id'], 
                        title='RFM Segmentation Scatter Plot')
fig_bubble.update_layout(xaxis_type="log")
fig_bubble.update_layout(yaxis_type="log")
fig_bubble.show()

In [36]:
fig_counts = px.histogram(client_agg_df, 
                          x='Segment', 
                          title='Number of Client per Segment', 
                          color='Segment', 
                          text_auto=True)
fig_counts.show()

In [ ]:
""" Most clients are Mid-Tier, while a notable share are At Risk
--> This shows a stable customer base but highlights the need to reengage those losing interest and nurture potential loyalists
"""

In [38]:
client_agg_df[['client_id', 'R_Score', 'F_Score', 'M_Score', 'Segment']].to_csv(
    "client_segments.csv", index=False)